In [1]:
import pandas as pd
import numpy as np
import psycopg2
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
pd.set_option('mode.chained_assignment', None)

In [2]:
conn = psycopg2.connect("dbname=nhl_db")
cur = conn.cursor()

In [3]:
cur.execute("""
    SELECT
        team,
        season,
        game_date,
        win,
        loss,
        ot_loss,
        goals_for,
        goals_against
    FROM summary
    WHERE playoffs = FALSE
    AND season >= '2012/13'
""")

In [4]:
df = pd.DataFrame(data=cur.fetchall(), columns=["team", "season", "date", "wins", "losses", "ot_losses", "goals_for", "goals_against"])

In [5]:
df.head()

,team,season,date,wins,losses,ot_losses,goals_for,goals_against
0,BOS,2021/22,2022-03-12,1,0,0,3,2
1,BOS,2021/22,2022-01-08,1,0,0,5,2
2,OTT,2021/22,2021-12-11,1,0,0,4,0
3,OTT,2021/22,2022-04-29,1,0,0,4,2
4,OTT,2021/22,2021-10-29,1,0,0,4,1


In [6]:
conn.close()

In [7]:
season_df = df.groupby(["team", "season"])[["wins", "losses", "ot_losses", "goals_for", "goals_against"]].sum().reset_index()

In [8]:
season_df["games_played"] = season_df["wins"] + season_df["losses"] + season_df["ot_losses"]
season_df["losses"] = season_df["losses"] + season_df["ot_losses"]
season_df.drop("ot_losses", axis=1)

,team,season,wins,losses,goals_for,goals_against,games_played
0,ANA,2012/13,30,18,134,115,48
1,ANA,2013/14,54,28,263,203,82
2,ANA,2014/15,51,31,228,221,82
3,ANA,2015/16,46,36,215,188,82
4,ANA,2016/17,46,36,220,197,82
...,...,...,...,...,...,...,...
301,WSH,2017/18,49,33,256,238,82
302,WSH,2018/19,48,34,274,248,82
303,WSH,2019/20,41,28,236,212,69
304,WSH,2020/21,36,20,188,161,56


In [9]:
season_df.head(20)

,team,season,wins,losses,ot_losses,goals_for,goals_against,games_played
0,ANA,2012/13,30,18,6,134,115,48
1,ANA,2013/14,54,28,8,263,203,82
2,ANA,2014/15,51,31,7,228,221,82
3,ANA,2015/16,46,36,11,215,188,82
4,ANA,2016/17,46,36,13,220,197,82
5,ANA,2017/18,44,38,13,231,209,82
6,ANA,2018/19,35,47,10,196,248,82
7,ANA,2019/20,29,42,9,182,225,71
8,ANA,2020/21,17,39,9,124,177,56
9,ANA,2021/22,31,51,14,228,266,82


In [10]:
season_df["win_pc"] = season_df["wins"] / season_df["games_played"]

In [11]:
season_df.head()

,team,season,wins,losses,ot_losses,goals_for,goals_against,games_played,win_pc
0,ANA,2012/13,30,18,6,134,115,48,0.625000
1,ANA,2013/14,54,28,8,263,203,82,0.658537
2,ANA,2014/15,51,31,7,228,221,82,0.621951
3,ANA,2015/16,46,36,11,215,188,82,0.560976
4,ANA,2016/17,46,36,13,220,197,82,0.560976


In [12]:
def pythagoreanExpectation(row, alpha=2):
    return pow(row.goals_for,alpha) / (pow(row.goals_for,alpha) + pow(row.goals_against,alpha))

In [13]:
# season_df["pyth_exp"] = season_df.apply(lambda x: pythagoreanExpectation(x, alpha=1.7), axis=1)

In [14]:
season_df.head()

,team,season,wins,losses,ot_losses,goals_for,goals_against,games_played,win_pc
0,ANA,2012/13,30,18,6,134,115,48,0.625000
1,ANA,2013/14,54,28,8,263,203,82,0.658537
2,ANA,2014/15,51,31,7,228,221,82,0.621951
3,ANA,2015/16,46,36,11,215,188,82,0.560976
4,ANA,2016/17,46,36,13,220,197,82,0.560976


In [15]:
seasons = season_df.season.unique()
teams = season_df.team.unique()

In [16]:
league = pd.DataFrame(columns=["season", "alpha", "mse"])

In [17]:
# for season in seasons:
#     df = season_df.loc[season_df.season == season, :]
    
#     def f(alpha):
#         df["pyth_exp"] = df.apply(lambda x: pythagoreanExpectation(x, alpha=alpha), axis=1)
    
#         mse = mean_squared_error(df["win_pc"], df["pyth_exp"])

#         return mse
    
#     res = minimize(f, x0=2.0)

In [18]:
def optimizePythagoreanExpectation(group, column_name):
    def f(alpha):
        pyth_exp = group.apply(lambda x: pythagoreanExpectation(x, alpha=alpha), axis=1)
    
        mse = mean_squared_error(group["win_pc"], pyth_exp)

        return mse
    
    res = minimize(f, x0=2.0)
    group[column_name] = res.x[0]

    return group

In [19]:
season_df = season_df.groupby(["season"], group_keys=True).apply(lambda x: optimizePythagoreanExpectation(x, "league_pe_alpha"))
season_df.reset_index(drop = True, inplace = True)

In [20]:
season_df = season_df.groupby(["season", "team"], group_keys=True).apply(lambda x: optimizePythagoreanExpectation(x, "team_pe_alpha"))
season_df.reset_index(drop = True, inplace = True)

In [21]:
season_df.head(20)

,team,season,wins,losses,ot_losses,goals_for,goals_against,games_played,win_pc,league_pe_alpha,team_pe_alpha
0,ANA,2012/13,30,18,6,134,115,48,0.625000,2.2813,3.340515
1,ARI,2012/13,21,27,9,121,125,48,0.437500,2.2813,7.686881
2,BOS,2012/13,28,20,6,127,106,48,0.583333,2.2813,1.860936
3,BUF,2012/13,21,27,6,118,139,48,0.437500,2.2813,1.533952
4,CAR,2012/13,19,29,4,127,159,48,0.395833,2.2813,1.881168
5,CBJ,2012/13,24,24,7,115,115,48,0.500000,2.2813,2.000000
6,CGY,2012/13,19,29,4,128,157,48,0.395833,2.2813,2.070601
7,CHI,2012/13,36,12,5,149,97,48,0.750000,2.2813,2.559371
8,COL,2012/13,16,32,7,114,150,48,0.333333,2.2813,2.525318
9,DAL,2012/13,22,26,4,128,141,48,0.458333,2.2813,1.726410
